In [3]:
from google.colab import drive
drive._mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [2]:
%tensorflow_version 1.x 

TensorFlow 1.x selected.


In [4]:
!pip install transformers
!pip install -U sentence-transformers

     |████████████████████████████████| 3.4 MB 5.4 MB/s 
     |████████████████████████████████| 596 kB 46.0 MB/s 
     |████████████████████████████████| 61 kB 509 kB/s 
     |████████████████████████████████| 895 kB 47.3 MB/s 
     |████████████████████████████████| 3.3 MB 44.8 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
     |████████████████████████████████| 78 kB 3.4 MB/s 
     |████████████████████████████████| 1.2 MB 11.2 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.1.0-py3-none-any.whl size=121000 sha256=90978bbbda70a63caaa5c19400d61b024a5cdfca3c8f7a6f3a622f0efb325ab9
  Stored in directory: /root/.cache/pip/wheels/90/f0/bb/ed1add84da70092ea526466eadc2bfb197c4bcb8d4fa5f7bad
Successfully built sentence-transformers


In [5]:
%cd /content/drive/MyDrive/20211/rec-sys/mind-recomendation/naml

/content/drive/MyDrive/20211/rec-sys/mind-recomendation/naml


In [6]:
import sys
import os
import numpy as np
import zipfile
from tqdm import tqdm 
from tempfile import TemporaryDirectory
import tensorflow as tf
tf.get_logger().setLevel('ERROR') # only show error messages

from recommenders.models.deeprec.deeprec_utils import download_deeprec_resources 
from recommenders.models.newsrec.newsrec_utils import prepare_hparams
from recommenders.models.newsrec.models.naml_bert import NAMLModel
from recommenders.models.newsrec.io.mind_all_iterator import MINDAllIterator
from recommenders.models.newsrec.io.mind_all_iterator_not_dict import MINDAllIteratorNotDictTitle
from recommenders.models.newsrec.newsrec_utils import get_mind_data_set

print("System version: {}".format(sys.version))
print("Tensorflow version: {}".format(tf.__version__))


Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/420M [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/350 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/13.2k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

System version: 3.7.12 (default, Sep 10 2021, 00:21:48) 
[GCC 7.5.0]
Tensorflow version: 1.15.2


## Prepare Parameters

In [7]:
epochs = 3
seed = 42
batch_size = 32

# Options: demo, small, large
MIND_type = 'small'

## Download and load data

In [8]:
data_path = "/content/drive/MyDrive/20211/rec-sys/mind-recomendation/data"

train_news_file = os.path.join(data_path, MIND_type+'/train', r'news.tsv')
train_behaviors_file = os.path.join(data_path, MIND_type+'/train', r'behaviors.tsv')
valid_news_file = os.path.join(data_path,MIND_type+ '/valid', r'news.tsv')
valid_behaviors_file = os.path.join(data_path,MIND_type+ '/valid', r'behaviors.tsv')


wordEmb_file = os.path.join(data_path, "utils", "embedding.npy")
userDict_file = os.path.join(data_path, "utils", "uid2index.pkl")
wordDict_file = os.path.join(data_path, "utils", "word_dict.pkl")

# add cai nay cho naml 
vertDict_file = os.path.join(data_path, "utils", "vert_dict.pkl")
subvertDict_file = os.path.join(data_path, "utils", "subvert_dict.pkl")

yaml_file = os.path.join(data_path, "utils", r'naml.yaml')

mind_url, mind_train_dataset, mind_dev_dataset, mind_utils = get_mind_data_set(MIND_type)

if not os.path.exists(train_news_file):
    download_deeprec_resources(mind_url, os.path.join(data_path,MIND_type+ '/train'), mind_train_dataset)
    
if not os.path.exists(valid_news_file):
    download_deeprec_resources(mind_url, \
                               os.path.join(data_path,MIND_type+ '/valid'), mind_dev_dataset)
if not os.path.exists(yaml_file):
    download_deeprec_resources(r'https://recodatasets.z20.web.core.windows.net/newsrec/', \
                               os.path.join(data_path, 'utils'), mind_utils)

## Create hyper-parameters

In [9]:
hparams = prepare_hparams(yaml_file, 
                          wordEmb_file=wordEmb_file,
                          wordDict_file=wordDict_file, 
                          userDict_file=userDict_file,
                          vertDict_file=vertDict_file, 
                          subvertDict_file=subvertDict_file,
                          batch_size=batch_size,
                          epochs=epochs)
print(hparams)

HParams object with values {'support_quick_scoring': True, 'dropout': 0.2, 'attention_hidden_dim': 200, 'head_num': 4, 'head_dim': 100, 'filter_num': 400, 'window_size': 3, 'vert_emb_dim': 100, 'subvert_emb_dim': 100, 'gru_unit': 400, 'type': 'ini', 'user_emb_dim': 50, 'learning_rate': 0.0001, 'optimizer': 'adam', 'epochs': 3, 'batch_size': 32, 'show_step': 100000, 'title_size': 768, 'body_size': 50, 'his_size': 50, 'vert_num': 17, 'subvert_num': 249, 'data_format': 'naml', 'npratio': 4, 'metrics': ['group_auc', 'mean_mrr', 'ndcg@5;10'], 'word_emb_dim': 300, 'cnn_activation': 'relu', 'model_type': 'naml', 'dense_activation': 'relu', 'loss': 'cross_entropy_loss', 'wordEmb_file': '/content/drive/MyDrive/20211/rec-sys/mind-recomendation/data/utils/embedding.npy', 'wordDict_file': '/content/drive/MyDrive/20211/rec-sys/mind-recomendation/data/utils/word_dict.pkl', 'userDict_file': '/content/drive/MyDrive/20211/rec-sys/mind-recomendation/data/utils/uid2index.pkl', 'vertDict_file': '/content/

In [10]:
# iterator = MINDAllIterator(hparams)  # mind all thì có thêm load cả body nữa 

In [11]:
print(train_news_file)
print(train_behaviors_file)

/content/drive/MyDrive/20211/rec-sys/mind-recomendation/data/small/train/news.tsv
/content/drive/MyDrive/20211/rec-sys/mind-recomendation/data/small/train/behaviors.tsv


In [12]:
# iterator.load_data_from_file(news_file= train_news_file,behavior_file= train_behaviors_file)

In [13]:
# import pickle
# with open(wordDict_file, "rb") as f:
#     a = pickle.load(f)

In [14]:
# len(a)  # lúc load cái từ lên nó đã cho qua một từ điển 60992 từ rồi vcl 

In [15]:
# iterator = MINDAllIterator

iterator = MINDAllIteratorNotDictTitle

## Train the NAML model

In [16]:
model = NAMLModel(hparams, iterator, seed=seed)

build dense
(?, 768)
<class 'tensorflow.python.framework.ops.Tensor'>
(?, 5, 400) (?, 400)


In [17]:
model.model.summary()


Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            [(None, 5, 768)]     0                                            
__________________________________________________________________________________________________
input_6 (InputLayer)            [(None, 5, 50)]      0                                            
__________________________________________________________________________________________________
input_7 (InputLayer)            [(None, 5, 1)]       0                                            
__________________________________________________________________________________________________
input_8 (InputLayer)            [(None, 5, 1)]       0                                            
______________________________________________________________________________________________

In [18]:
model._build_naml()[0].summary()

build dense
(?, 768)
<class 'tensorflow.python.framework.ops.Tensor'>
(?, 5, 400) (?, 400)
Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_23 (InputLayer)           [(None, 5, 768)]     0                                            
__________________________________________________________________________________________________
input_24 (InputLayer)           [(None, 5, 50)]      0                                            
__________________________________________________________________________________________________
input_25 (InputLayer)           [(None, 5, 1)]       0                                            
__________________________________________________________________________________________________
input_26 (InputLayer)           [(None, 5, 1)]       0                                            
_

In [19]:
# print(model.run_eval(valid_news_file, valid_behaviors_file))

In [ ]:
%%time
model.fit(train_news_file, train_behaviors_file,valid_news_file, valid_behaviors_file)

0it [00:00, ?it/s]

title shape 51283 


7385it [2:07:44,  1.04s/it]
0it [00:00, ?it/s]

title shape 42417 


42386it [1:12:12,  9.78it/s] 
73121it [07:45, 157.08it/s]
73152it [00:14, 5001.77it/s]


at epoch 1
train info: logloss loss:1.4043879541014204
eval info: group_auc:0.5344, mean_mrr:0.244, ndcg@10:0.3129, ndcg@5:0.2506
at epoch 1 , train time: 7664.6 eval time: 4905.4


1311it [07:38,  2.89it/s]

In [ ]:
%%time
# res_syn = model.run_eval(valid_news_file, valid_behaviors_file)
# print(res_syn)

In [ ]:
# sb.glue("res_syn", res_syn)

## Save the model

In [ ]:
model_path = os.path.join(data_path, "model-naml-lstm")
os.makedirs(model_path, exist_ok=True)

model.model.save_weights(os.path.join(model_path, "naml_ckpt"))

## Output Prediction File
This code segment is used to generate the prediction.zip file, which is in the same format in [MIND Competition Submission Tutorial](https://competitions.codalab.org/competitions/24122#learn_the_details-submission-guidelines).

Please change the `MIND_type` parameter to `large` if you want to submit your prediction to [MIND Competition](https://msnews.github.io/competition.html).

In [ ]:
group_impr_indexes, group_labels, group_preds = model.run_fast_eval(valid_news_file, valid_behaviors_file)

In [ ]:
with open(os.path.join(data_path, 'prediction.txt'), 'w') as f:
    for impr_index, preds in tqdm(zip(group_impr_indexes, group_preds)):
        impr_index += 1
        pred_rank = (np.argsort(np.argsort(preds)[::-1]) + 1).tolist()
        pred_rank = '[' + ','.join([str(i) for i in pred_rank]) + ']'
        f.write(' '.join([str(impr_index), pred_rank])+ '\n')

In [ ]:
f = zipfile.ZipFile(os.path.join(data_path, 'prediction.zip'), 'w', zipfile.ZIP_DEFLATED)
f.write(os.path.join(data_path, 'prediction.txt'), arcname='prediction.txt')
f.close()

## Reference
\[1\] Chuhan Wu, Fangzhao Wu, Mingxiao An, Jianqiang Huang, Yongfeng Huang and Xing Xie: Neural News Recommendation with Attentive Multi-View Learning, IJCAI 2019<br>
\[2\] Wu, Fangzhao, et al. "MIND: A Large-scale Dataset for News Recommendation" Proceedings of the 58th Annual Meeting of the Association for Computational Linguistics. https://msnews.github.io/competition.html <br>
\[3\] GloVe: Global Vectors for Word Representation. https://nlp.stanford.edu/projects/glove/